In [1]:
!git clone https://github.com/mohrezaeib/maml-bert.git

fatal: destination path 'maml-bert' already exists and is not an empty directory.


In [2]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import pandas as pd 
import os
DATA_PATH = "./maml-bert/data/raw/ph.csv"
df = pd.read_csv(DATA_PATH, index_col=None)
df.iloc[:10]['seq']

0    MAVEPLRVLVTGAAGQIGYALVPMIARGIMLGANQPVILHMLDIPP...
1    MKVAVLGAAGGIGQALALLLKTQLPAGSELSLYDIAPVTPGVAVDL...
2    MFATRQAFNLFQKRAFSASARQSSKVAILGAAGGIGQPLSLLMKLN...
3    MFATRQAFNLFQKRAFSASARQSSKVAILGAAGGIGQPLSLLMKLN...
4    MKLGFVGAGRVGSTSAFTCLLNLDVDEIALVDIAEDLAVGEAMDLA...
5    MPGPLRVLITGAAGQIAYNLSNMVANGNLFGKDQKIILHLLDIPEA...
6    MKTPVKIAVTGAAGQISYSLLFRLASGELLGPDQPMIFHLLETPQA...
7    MITIVGSGRVGTAAAAIMGIMRIDKKILLIDIVKGLPQGEALDLNH...
8    MKAPVRVAVTGAAGQIGYSLLFRIAAGEMLGKDQPVILQLLEIPQA...
9    MAPPQTIPRRGLFIGGAWREPCLGRRLPVVNPATEATIGDIPAGTA...
Name: seq, dtype: object

In [4]:
!pip install torch-optimizer


In [5]:
import pandas as pd 
import os
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn.functional as F
import torch.nn as nn
# from __future__ import print_function
import argparse
import json
import logging
import os
import sys

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader, RandomSampler, TensorDataset
from transformers import BertTokenizer, get_linear_schedule_with_warmup
import torch_optimizer as optim

import torch
from torch import nn
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader, RandomSampler, TensorDataset


DATA_PATH = "./maml-bert/data/raw/ph.csv"
df = pd.read_csv(DATA_PATH, index_col=None)
# df = df[:10]



class ProteinSequenceDataset(Dataset):
    def __init__(self, sequence, targets, tokenizer, max_len):
        self.sequence = sequence
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sequence)

    def __getitem__(self, item):
        sequence = str(self.sequence[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            sequence,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
          'protein_sequence': sequence,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }



PRE_TRAINED_MODEL_NAME = 'Rostlab/prot_bert_bfd_localization'
class ProteinClassifier(nn.Module):
    def __init__(self, n_classes):
        super(ProteinClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                        nn.Linear(self.bert.config.hidden_size, n_classes),
                                        nn.Tanh())
        
    def forward(self, input_ids, attention_mask):
        output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        return self.classifier(output.pooler_output)



# Network definition
# from model_def import ProteinClassifier
# from data_prep import ProteinSequenceDataset
 
# ## SageMaker Distributed code.
# from smdistributed.dataparallel.torch.parallel.distributed import DistributedDataParallel as DDP
# import smdistributed.dataparallel.torch.distributed as dist

# dist.init_process_group()

logger = logging.getLogger(__name__)
# logger.setLevel(logging.DEBUG)
# logger.addHandler(logging.StreamHandler(sys.stdout))

MAX_LEN = 512  # this is the max length of the sequence
PRE_TRAINED_MODEL_NAME = 'Rostlab/prot_bert_bfd_localization'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, do_lower_case=False)

def _get_train_data_loader(batch_size, training_dir):
    # dataset = pd.read_csv(os.path.join(training_dir, "deeploc_per_protein_train.csv"))
    dataset = df[:400]
    train_data = ProteinSequenceDataset(
        sequence=dataset.seq.to_numpy(),
        targets=dataset.label.to_numpy(),
        tokenizer=tokenizer,
        max_len=MAX_LEN
  )

    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True
                                  )
    return train_dataloader

def _get_test_data_loader(batch_size, training_dir):
    # dataset = pd.read_csv(os.path.join(training_dir, "deeploc_per_protein_test.csv"))
    dataset = df[400:]
    test_data = ProteinSequenceDataset(
        sequence=dataset.seq.to_numpy(),
        targets=dataset.label.to_numpy(),
        tokenizer=tokenizer,
        max_len=MAX_LEN
  )
    test_sampler = RandomSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
    return test_dataloader

def freeze(model, frozen_layers):
    modules = [model.bert.encoder.layer[:frozen_layers]] 
    for module in modules:
        for param in module.parameters():
            param.requires_grad = False
            
def train(args):
    use_cuda = args.num_gpus > 0
    device = torch.device("cuda" if use_cuda else "cpu")
    
    world_size = 1
    rank =0
    local_rank=0
#     rank = dist.get_rank()
#     local_rank = dist.get_local_rank()
    
    # set the seed for generating random numbers
    torch.manual_seed(args.seed)
    if use_cuda:
        torch.cuda.manual_seed(args.seed)

    train_loader = _get_train_data_loader(args.batch_size, args.data_dir)
    if rank == 0:
        test_loader = _get_test_data_loader(args.test_batch_size, args.test)
        print("Max length of sequence: ", MAX_LEN)
        print("Freezing {} layers".format(args.frozen_layers))
        print("Model used: ", PRE_TRAINED_MODEL_NAME)

    logger.debug(
        "Processes {}/{} ({:.0f}%) of train data".format(
            len(train_loader.sampler),
            len(train_loader.dataset),
            100.0 * len(train_loader.sampler) / len(train_loader.dataset),
        )
    )

    model = ProteinClassifier(
        args.num_labels  # The number of output labels.
    )
    freeze(model, args.frozen_layers)
    model =model.to(device)
#     torch.cuda.set_device(local_rank)
#     model.cuda(local_rank)
    
    optimizer = optim.Lamb(
            model.parameters(), 
            lr = args.lr , 
            betas=(0.9, 0.999), 
            eps=args.epsilon, 
            weight_decay=args.weight_decay)
    
    total_steps = len(train_loader.dataset)
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps)
    
    loss_fn = nn.CrossEntropyLoss().to(device)
    
    for epoch in range(1, args.epochs + 1):
        model.train()
        for step, batch in enumerate(train_loader):
            b_input_ids = batch['input_ids'].to(device)
            b_input_mask = batch['attention_mask'].to(device)
            b_labels = batch['targets'].to(device)

            outputs = model(b_input_ids,attention_mask=b_input_mask)
            loss = loss_fn(outputs, b_labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()
            optimizer.zero_grad()
            
#             if step % args.log_interval == 0 :
            logger.info(
                "Collecting data from Master Node: \n Train Epoch: {} [{}/{} ({:.0f}%)] Training Loss: {:.6f}".format(
                    epoch,
                    step * len(batch['input_ids'])*world_size,
                    len(train_loader.dataset),
                    100.0 * step / len(train_loader),
                    loss.item(),
                )
            )
            if  step % 100==0:
                print('Batch', step, "from rank", rank)
        test(model, test_loader, device)
        scheduler.step()
    
    model_save = model.module if hasattr(model, "module") else model
    save_model(model_save, "./")

def save_model(model, model_dir):
    path = os.path.join(model_dir, 'model.pth')
    # recommended way from http://pytorch.org/docs/master/notes/serialization.html
    torch.save(model.state_dict(), path)
    logger.info(f"Saving model: {path} \n")

def test(model, test_loader, device):
    model.eval()
    losses = []
    correct_predictions = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    tmp_eval_accuracy, eval_accuracy = 0, 0
    
    with torch.no_grad():
        for batch in test_loader:
            b_input_ids = batch['input_ids'].to(device)
            b_input_mask = batch['attention_mask'].to(device)
            b_labels = batch['targets'].to(device)

            outputs = model(b_input_ids,attention_mask=b_input_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, b_labels)
            correct_predictions += torch.sum(preds == b_labels)
            losses.append(loss.item())
            
    print('\nTest set: Validation loss: {:.4f}, Validation Accuracy: {:.0f}%\n'.format(
        np.mean(losses),
        100. * correct_predictions.double() / len(test_loader.dataset)))






ImportError: cannot import name 'get_linear_schedule_with_warmup' from 'transformers' (c:\Users\mohre\anaconda3\lib\site-packages\transformers\__init__.py)

In [ ]:
class Arg(object):
    pass

args = Arg()
args.num_labels =2
args.batch_size=1
args.test_batch_size = 8
args.epochs = 12
args.lr = 0.3e-5
args.weight_decay = 0.01
args.seed = 43
args.epsilon = 1e-8
args.frozen_layers = 10
args.verbose =True
args.use_cuda = True
args.num_gpus = 1
args.data_dir =""
args.test ="test"
args.log_interval=10



In [ ]:
train(args)

Max length of sequence:  512
Freezing 10 layers
Model used:  Rostlab/prot_bert_bfd_localization


Downloading:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_bert_bfd_localization were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batch 0 from rank 0
Batch 100 from rank 0
Batch 200 from rank 0
Batch 300 from rank 0

Test set: Validation loss: 0.5798, Validation Accuracy: 88%

Batch 0 from rank 0
Batch 100 from rank 0
Batch 200 from rank 0
Batch 300 from rank 0

Test set: Validation loss: 0.5526, Validation Accuracy: 88%

Batch 0 from rank 0
Batch 100 from rank 0
Batch 200 from rank 0
Batch 300 from rank 0

Test set: Validation loss: 0.6170, Validation Accuracy: 88%

Batch 0 from rank 0
Batch 100 from rank 0
Batch 200 from rank 0
Batch 300 from rank 0

Test set: Validation loss: 0.5259, Validation Accuracy: 88%

Batch 0 from rank 0
Batch 100 from rank 0
Batch 200 from rank 0
Batch 300 from rank 0

Test set: Validation loss: 0.5140, Validation Accuracy: 88%

Batch 0 from rank 0
Batch 100 from rank 0
Batch 200 from rank 0
Batch 300 from rank 0

Test set: Validation loss: 0.5054, Validation Accuracy: 88%

Batch 0 from rank 0
Batch 100 from rank 0
Batch 200 from rank 0
Batch 300 from rank 0

Test set: Validation loss

In [ ]:
!ls -lh

total 1.6G
---------- 1 root root  54K Jul 28 10:35 __notebook__.ipynb
drwxr-xr-x 4 root root 4.0K Jul 28 10:03 maml-bert
-rw-r--r-- 1 root root 1.6G Jul 28 10:35 model.pth
